# Parmi les visiteurs de pages CC combien utilisent la partie « articles de la convention collective »

**Objectif** : Parmi les visiteurs des pages CC (page qui contiennent «  /convention-collective/ » dans l’url) : 
- combien cliquent sur les accordéons qui sont dans la partie « Articles de la convention collective » : 
- combien cliquent sur les « tuiles articles » qui amènent sur légifrance : 


## On commence par le chargement des données

In [ ]:
import pandas as pd
import numpy as np
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict

tqdm.pandas()


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 200)

## Si on veut lire en requêtant Elastic Search

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
  "query": {
    "bool": { 
      "must": [
        {
          "prefix": {
            "url": "https://code.travail.gouv.fr/convention-collective/" 
          }
        },
        {
          "range": {
           "logfile": {
              "gte": "2022-07-01",
              "lt": "2023-11-01"
           }
          }
        }
      ]
    }
  }
}

In [ ]:
logs = es_connector.execute_query(QUERY, "logs-new")

# Début de l'exploration du funnel pour l'indemnite licenciement
## 1. Nombre de visites sur les pages convention collective

### Comptage du nombre de visiteurs uniques sur ces pages

In [ ]:
unique_visits = logs.idVisit.nunique()
unique_visits

## 2. Nombre de click sur un menu déroulant article de convention collective 

On va compter le nombre de click sur `pagecc_clicktheme`

In [ ]:
nb_clicks_pagecc_clicktheme = logs[
    (logs['type']=='pagecc_clicktheme')
].idVisit.nunique()
nb_clicks_pagecc_clicktheme

## 3. Events de sortie

Le problème c'est qu'on a pas les évents de sorties

### Première étape

In [ ]:
visits_on_contrib_without_idcc = logs_april_contrib_without_duplicates[
    logs_april_contrib_without_duplicates['url'].str.match('^https://code.travail.gouv.fr/contribution/[a-zA-Z]+', 
                                                     na=False)
]

In [ ]:
nb_total_visits_on_contrib_without_idcc = visits_on_contrib_without_idcc.type.value_counts()['visit_content']
nb_total_visits_on_contrib_without_idcc

### Deuxième étape

In [ ]:
cc_select_on_contrib_without_idcc = visits_on_contrib_without_idcc.type.value_counts()['cc_select']
cc_select_on_contrib_without_idcc

### Présentation des résultats

In [ ]:
print(f"{round(( cc_select_on_contrib_without_idcc) / nb_total_visits_on_contrib_without_idcc * 100 , 2)} %")

### Présentation Graphique du résultat